# Programming Assignment 2

In this assignment, you will:

1. Practice writing code to train and evaluate models using both the two-way holdout method, and an evaluation approach appropriate for models with hyperparameters that uses k-fold cross validation plus a test set.

2. Practice writing code to optimize a machine learning model. In particular, we will use gradient descent to optimize a logistic regression model.

3. **574 Only**: Perform optimization with a different algorithm (Newton-Raphson)

# Resources you can use to complete this assignment (a COMPLETE list)

**NOTE: You ARE allowed to use Google to find things that fit this list (i.e. it is often easy to google something like "plotly draw line graph" to find the right part of the plotly documentation).**

- Anything linked to in this article
- Anything linked to from the course web page
- Any materials from another online course taught at a university (**if you use this, you MUST provide a link to the exact document used**)
- Anything posted by Kenny, Navid, or Yincheng on Piazza



# Grading

For grading of code in Part 2, we will execute the submitted notebook as follows:

```shell
jupyter nbconvert --to python Assignment2-Student.ipynb
python Assignment2-Student.py
```

The PDF that comes along with this document has other details on the points awarded for each part.

As such, you will submit, one member of your group will subit as a zip file on UBLearns, a ```.zip``` file that contains 4 things:
- Your completed jupyter notebook.
- Your written report, answering all questions asked here (and copied in the assignment PDF)
- `part_1.1_results.csv`
- `part_1.4_results.csv`


# Part 1 - Predicting Review Scores on Pitchfork

For Part 1, we will be using data from [this paper](https://ojs.aaai.org/index.php/ICWSM/article/view/7355). The data is a collection of reviews from [Pitchfork](https://pitchfork.com/), a site that provides expert reviews of music album. The authors of this paper have also combined the data with a set of features from [Spotify’s API](https://developer.spotify.com/documentation/web-api/) that provide insight into the music itself, e.g. the "acousticness" of the song.  We will tackle a regression problem here, trying to predict the score of a review from several of the other columns in the dataset.

## Part 1.1 - Feature Engineering with Feature Subsets

In the first subsection of Part 1, we’re going to rely on our old friend linear regression. We’re going to look at how running linear regression with various subsets of our features impacts our ability to predict score.

In Part 1.1, your task is to write code below that trains a separate linear regression model for a number of different feature subsets.  Specifically:

- The list `feature_sets` below is a list of lists; each sublist is a different subset of features to build a model with. 
- All models should be trained on the dataset `part1_train.csv`. 
- For each of these trained models, you should evaluate the model’s predictions on the training dataset, as well as the provided test set, called `part1_test.csv`. The evaluation metric we will use is **root mean squared error**.  

Write out the result to a file called `part_1.1_results.csv` and submit this along with your assignment. The file should have the following columns:
- `feature_set` - a column describing the features of the model used. For feature sets with multiple features, combine them using an underscore (you can do this with the code `"_".join(feature_set)`)
- `training_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the training data
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data

In addition, please answer the following questions:
- **1.1.1** Which model had the best RMSE on the *training data*? 
- **1.1.2** Which model had the best RMSE on the *test data*? 
- **1.1.3** Which feature do you believe was the most important one? Why? *(Note: There is more than one perfectly acceptable way to answer this question)*
- **1.1.4** What can we say about the utility of the Spotify features based on these results?

In [1]:
feature_sets = [['artist'],
 ['reviewauthor'],
 ['releaseyear'],
 ['recordlabel'],
 ['genre'],
 ['danceability'],
 ['energy'],
 ['key'],
 ['loudness'],
 ['speechiness'],
 ['acousticness'],
 ['instrumentalness'],
 ['liveness'],
 ['valence'],
 ['tempo'],
 ['danceability','energy','key','loudness','speechiness','acousticness',
  'instrumentalness','liveness','valence','tempo'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre'],
 ['artist', 'reviewauthor', 'releaseyear', 'recordlabel', 'genre', 'danceability', 
  'energy', 'key', 'loudness', 'speechiness', 'acousticness', 'instrumentalness',
  'liveness', 'valence', 'tempo']]

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# Convenience things for you, note that releaseyear is continuous but is not a Spotify API variable
CONTINUOUS_FEATURES = ['releaseyear', 'danceability', 'energy', 'key', 'loudness',
       'speechiness', 'acousticness', 'instrumentalness', 'liveness',
       'valence', 'tempo']
CATEGORICAL_FEATURES = ['artist', 'reviewauthor', 'recordlabel', 'genre']

# Read in the data
training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")

training_data[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(training_data[CONTINUOUS_FEATURES].values)
test_data[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(test_data[CONTINUOUS_FEATURES].values)

for var in CATEGORICAL_FEATURES:
       training_data[var] = LabelEncoder().fit_transform(training_data[var])
       test_data[var] = LabelEncoder().fit_transform(test_data[var])

training_data = training_data.reset_index()
test_data = test_data.reset_index()

result_data = []
for feature_set in feature_sets:
       X_train = training_data[feature_set].to_numpy()
       X_test = test_data[feature_set].to_numpy()
       y_train = np.log(training_data['score'] + 1)
       y_test = np.log(test_data['score'] + 1)
       lr = LinearRegression()
       lr.fit(X_train,y_train)
       y_pred_train = lr.predict(X_train)
       y_pred_test = lr.predict(X_test)
       rmse_train = metrics.mean_squared_error(y_train, y_pred_train, squared=False)
       rmse_test = metrics.mean_squared_error(y_test, y_pred_test, squared=False)
       result_data.append(['_'.join(feature_set), rmse_train, rmse_test])
result_df = pd.DataFrame(result_data,columns=['feature_set', 'training_rmse', 'test_rmse'])
result_df.to_csv('part_1.1_results.csv')

## Part 1.2 - Feature Engineering with the LASSO

In Part 1.2, your task is to write code below that trains an L1-regularized linear regression model, with an expanded feature set.  Specifically:

1. Begin with the final feature set listed in `feature_sets` (i.e. your feature set, to begin this section, is `feature_sets[-1]`.
2. One-hot encode your categorical variables, setting `drop=if_binary` and `sparse=False` in the function arguments. 
3. Scale all of your continuous features using the `StandardScaler`.
4. Train an L1-regularized linear regression model using these features on the dataset `part1_train.csv`. You should use the [LassoCV](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html) class in `sklearn`, it will do the cross-validation necessary to select the appropriate value for the regularizer for you!  Use 10-fold cross-validation to perform model selection (set the `LassoCV` parmaeter `cv` to 10), and set the `random_state` to 1. Do not change any of the other parameters to `LassoCV` (i.e. leave them at their defaults).
5. Identify the best `alpha` value (the regularizer term, according to `sklearn`. In class, we refer to this as $\lambda$!) in terms of average mean squared error according to the cross-validation.
6. Finally, train a [Lasso](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) model on the entire training dataset (`part1_train.csv`). You will use this to report the root mean squared error on the test set (Question 1.2.4 below), and use it in Part 1.3 below as well.

**Hint: The proceedure outlined above is very similar to ones we have discussed in class and shown how to do using `Pipeline`s.** 


In [3]:
# Write your code for Part 1.2 here
from sklearn.linear_model import LassoCV, Lasso
from sklearn.preprocessing import OneHotEncoder

training_data = pd.read_csv("part1_train.csv")
test_data = pd.read_csv("part1_test.csv")
frames = [training_data, test_data]
concat_df = pd.concat(frames)

concat_df[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(concat_df[CONTINUOUS_FEATURES].values)

encoder = OneHotEncoder(drop = "if_binary", sparse=False)
encoded_concat_df_data = pd.DataFrame(encoder.fit_transform(concat_df[CATEGORICAL_FEATURES]))
feature_names = encoder.categories_

concat_df = concat_df.join(encoded_concat_df_data)

concat_df = concat_df.drop(CATEGORICAL_FEATURES, axis = 1)

training_data = concat_df.head(len(concat_df)-1808)
test_data = concat_df.tail(1808)

# Do the CV to find alpha
features = feature_sets[-1]

y_train = np.log(training_data['score'] + 1)
y_test = np.log(test_data['score'] + 1)

training_data = training_data.drop(['score'], axis = 1)
test_data = test_data.drop(['score'], axis = 1)

X_train = training_data.to_numpy()
X_test = test_data.to_numpy()

# Retrain the model
lassocv = LassoCV(random_state=1, cv=10).fit(X_train, y_train)

lasso = Lasso(alpha=lassocv.alpha_, random_state=1).fit(X_train, y_train)
y_pred_train = lasso.predict(X_train)
y_pred_test = lasso.predict(X_test)

len_encoded_columns = 0
for ele in feature_names:
    len_encoded_columns += len(ele)

print('1.2.1 Total number of features intorduced by Step 2 are ' + str(len_encoded_columns) + ' features, replacing ' + str(len(CATEGORICAL_FEATURES)) + ' features')
print('1.2.2 Best alpha value: ' + str(lassocv.alpha_))
rmse_train = metrics.mean_squared_error(y_train, y_pred_train, squared=False)
print('1.2.3 Average RMSE of train_data: ' + str(round(rmse_train, 4)))
rmse_test = metrics.mean_squared_error(y_test, y_pred_test, squared=False)
print('1.2.4 RMSE of test_data: ' + str(round(rmse_test, 4)))

1.2.1 Total number of features intorduced by Step 2 are 680 features, replacing 4 features
1.2.2 Best alpha value: 3.4212169735207094e-05
1.2.3 Average RMSE of train_data: 0.1746
1.2.4 RMSE of test_data: 0.1762


Now, answer the following questions:
- **1.2.1** - How many total features are introduced by Step 2 above? Provide both the number and an explanation of how you got to this number.
- **1.2.2** - What was the best `alpha` value according to your cross-validation results?
- **1.2.3** - What was the **average RMSE** of the model with this `alpha` value on the k-fold cross validation on the *training* data?
- **1.2.4** - What was the **RMSE** of the model with this `alpha` value on the k-fold cross validation on the *test* data?


# Part 1.3 - Interpreting Model Coefficents

In this section we will interpret the coefficients from the final model you trained on all of the training data.

- **1.3.1** - How many non-zero coefficients are in this final model?
- **1.3.2** - What percentage of the coefficients are non-zero in this final model?
- **1.3.3** - Who were the three most critical review authors, as estimated by the model? How do you know?
- **1.3.4** - Who were the three artists that reviewers tended to like the most?  How do you know?
- **1.3.5** - What genre did Pitchfork reviewers tend to like the most? Which genre did they like the least?


In [4]:
# 1.3.1
print('1.3.1 Number of non-zero coefficients in the final model: ' + str(np.count_nonzero(lasso.coef_)))
# 1.3.2
percentage_nonzero = (np.count_nonzero(lasso.coef_)/len(lasso.coef_))*100
print('1.3.2 Percentage of non-zero coefficients in the final model: ' + str(round(percentage_nonzero, 2)) + '%')
# 1.3.3
print('1.3.3:')
coef_auth = lasso.coef_[117:]
coef_auth = coef_auth[:len(coef_auth)-330]
display(pd.DataFrame(pd.DataFrame([list(l) for l in zip(feature_names[1], coef_auth)], columns=['Author', 'Coeff']).sort_values(by='Coeff', ascending=True).reset_index()[['Author', 'Coeff']].head(3)))
print('1.3.4:')
coef_artist = lasso.coef_[11:]
coef_artist = coef_artist[:len(coef_artist)-574]
display(pd.DataFrame(pd.DataFrame([list(l) for l in zip(feature_names[0], coef_artist)], columns=['Artist', 'Coeff']).sort_values(by='Coeff', ascending=False).reset_index()[['Artist', 'Coeff']].head(3)))
print('1.3.5:')
print('Users tend to like the most:')
coef_genre = lasso.coef_[-11:]
display(pd.DataFrame(pd.DataFrame([list(l) for l in zip(feature_names[3], coef_genre)], columns=['Genre', 'Coeff']).sort_values(by='Coeff', ascending=False).reset_index()[['Genre', 'Coeff']].head(3)))
print('Users tend to like the least:')
display(pd.DataFrame(pd.DataFrame([list(l) for l in zip(feature_names[3], coef_genre)], columns=['Genre', 'Coeff']).sort_values(by='Coeff', ascending=True).reset_index()[['Genre', 'Coeff']].head(3)))

1.3.1 Number of non-zero coefficients in the final model: 397
1.3.2 Percentage of non-zero coefficients in the final model: 57.45%
1.3.3:


,Author,Coeff
0,Alison Fields,-0.239852
1,Brian James,-0.204856
2,Mark Martelli,-0.186225


1.3.4:


,Artist,Coeff
0,R.E.M.,0.091655
1,Miles Davis,0.074612
2,Deerhoof,0.058392


1.3.5:
Users tend to like the most:


,Genre,Coeff
0,Jazz,0.007220
1,Global,0.005753
2,Pop/R&B,0.004058


Users tend to like the least:


,Genre,Coeff
0,NaN,-0.035546
1,Electronic,-0.026255
2,Metal,-0.024287


# Part 1.4 - "Manual" Cross-Validation + Holdout for Model Selection and Evaluation

We will finally use cross validation for both algorithm and model selection, with a hold-out test set for a final evaluation. We will use **5-fold cross validation** to identify the best parameters and hyperparameters for a set of models. We will then take our final models and use a final hold-out test set (the same one as above) to estimate the generalization error of the models.

Specifically, your task is first to write code that trains and evaluates the following models, one for each of the specified hyper parameters sets:

- `Decision Tree regression` - All combinations of a `max_depth` of 5, 10, or 20, and a `criterion` of `"squared error"` or `"absolute error"`. Use the [DecisionTreeRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html#sklearn.tree.DecisionTreeRegressor).
- Ridge regression - Use the following choices of L2 penalty: $[10^{-5}, 10^{-4}, ..., 10^4, 10^5]$. In Python, you can create a list of these numbers using `np.logspace(-5, 5, 11)`. Use the [Ridge](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html#sklearn.linear_model.Ridge) class from sklearn to train a Ridge Regression model. The parameters you need to pass when constructing the Ridge model are `alpha`, which lets you specify what you want the L2 penalty to be, and `random_state=0` to avoid randomness.
- kNN regression - Values of `n_neighbors` of 1, 5, 10, and 15. Use the [KNeighborsRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html) class.

Additional notes:
1. All models should use the feature sets described in Part 1.3 (the same ...one-hot encoded... categorical variables, and the scaled continuous variables)
2. As opposed to using the `KFold` class from `sklearn` like we did in class, we have instead provided you with pre-existing data sets; you should therefore use the pre-split data in the provided CSV files ``1.2_fold0.csv ... 1.2_fold4.csv``

**What to submit**:

1. Write out the result to a file called `part_1.4_results.csv` and submit this along with your assignment. The file should have the following columns:
- `model_name` - The name of the model, one of `DTR` (Decision Tree Regression), `Ridge`, or `KNN`.
- `hyperparameter_setting` - a column describing the hyperparameters of the model. For models with multiple hyperparameters, combine them using an underscore (you can do this with the code `"_".join(hyperparameters)`).
- `mean_training_rmse` - a column that gives the mean RMSE on the k-fold training data. You should take the average of the model’s errors on the different folds, using root mean squared error again as your evaluation metric.
- `sd_training_rmse` - a column that gives the standard deviation RMSE on the k-fold training data.
- `test_rmse` - a column that gives the RMSE of a linear regression model trained on this feature set on the test data


2. Answer the following questions:
- **1.4.1** Report, for each model, the hyper parameter setting that resulted in the best performance
- **1.4.2** Which model performed the best overall on the cross-validation? 
- **1.4.3** Which model performed the best overall on the final test set? 
- **1.4.4** With respect to your answer for 1.4.3, why do you think that might be? (*Note: there is more than one correct way to answer this question*)
- **1.4.5** Which model/hyperparameter setting had the highest standard deviation across the different folds of the cross validation?
- **1.4.6** With respect to your answer for 1.4.6, why do you think that might be? (*Note: there is more than one correct way to answer this question*)

In [5]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.neighbors import KNeighborsRegressor
frames = []
test_data = pd.read_csv('part1_test.csv')
frames.append(test_data)
for i in range(0, 5):
    filename = '1.2_fold' + str(i) + '.csv'
    train_data = pd.read_csv(filename)
    frames.append(train_data)
concat_df = pd.concat(frames)
concat_df[CONTINUOUS_FEATURES] = StandardScaler().fit_transform(concat_df[CONTINUOUS_FEATURES].values)
encoder = OneHotEncoder(drop = "if_binary", sparse=False)
encoded_concat_df_data = pd.DataFrame(encoder.fit_transform(concat_df[CATEGORICAL_FEATURES]))
feature_names = encoder.categories_
concat_df = concat_df.join(encoded_concat_df_data)
concat_df = concat_df.drop(CATEGORICAL_FEATURES, axis = 1)
test_data = concat_df.head(1808)
training_data = concat_df.tail(len(concat_df)-1808)
train_1, train_2, train_3, train_4, train_5 = np.array_split(training_data, 5)
train_dfs = [train_1, train_2, train_3, train_4, train_5]
result_arr = []
for fold in range(len(train_dfs)):
    y_train = np.log(train_dfs[fold]['score'] + 1)
    train_dfs[fold] = train_dfs[fold].drop(['score'], axis = 1)
    X_train = train_dfs[fold].to_numpy()
    # Descision Tree Regressor
    for depth in [5, 10, 20]:
        for j in ['squared_error', 'absolute_error']:
            DT_model = DecisionTreeRegressor(criterion=j, max_depth=depth).fit(X_train, y_train)
            y_pred_train = DT_model.predict(X_train)
            y_pred_test = DT_model.predict(X_test)
            rmse_train = metrics.mean_squared_error(y_train, y_pred_train, squared=False)
            rmse_test = metrics.mean_squared_error(y_test, y_pred_test, squared=False)
            result_arr.append([fold, 'DTR', str(depth) + '_' + str(j), rmse_train, rmse_test])
    for alpha in np.logspace(-5, 5, 11):
        RIDGE_model = Ridge(alpha = alpha, random_state=0).fit(X_train, y_train)
        y_pred_train = RIDGE_model.predict(X_train)
        y_pred_test = RIDGE_model.predict(X_test)
        rmse_train = metrics.mean_squared_error(y_train, y_pred_train, squared=False)
        rmse_test = metrics.mean_squared_error(y_test, y_pred_test, squared=False)
        result_arr.append([fold, 'Ridge', str(alpha), rmse_train, rmse_test])
    for n_neighbours in [1, 5, 10, 15]:
        KNN_model = KNeighborsRegressor(n_neighbors=n_neighbours).fit(X_train, y_train)
        y_pred_train = RIDGE_model.predict(X_train)
        y_pred_test = RIDGE_model.predict(X_test)
        rmse_train = metrics.mean_squared_error(y_train, y_pred_train, squared=False)
        rmse_test = metrics.mean_squared_error(y_test, y_pred_test, squared=False)
        result_arr.append([fold, 'KNN', str(n_neighbours), rmse_train, rmse_test])
result_df = pd.DataFrame(result_arr, columns=['fold', 'model_name', 'hyperparameter_setting', 'training_rmse', 'test_rmse'])
result_df = result_df.groupby(['hyperparameter_setting'], as_index=False).agg({'training_rmse': ['mean', 'std'], 'model_name': 'first', 'test_rmse': 'last'}).reset_index(drop=True)
result_df.columns = result_df.columns.get_level_values(1)
dict = {result_df.columns[0]: 'hyperparameter_setting',
        result_df.columns[1]: 'mean_training_rmse',
        result_df.columns[2]: 'std_training_rmse',
        result_df.columns[3]: 'model_name',
        result_df.columns[4]: 'test_rmse'}
result_df.rename(columns=dict, inplace=True)
result_df = result_df[['model_name', 'hyperparameter_setting', 'mean_training_rmse', 'std_training_rmse', 'test_rmse']]
custom_dict = {'DTR': 0, 'Ridge': 1, 'KNN': 2} 
result_df = result_df.sort_values(by=['model_name'], key=lambda x: x.map(custom_dict)).reset_index(drop=True)
result_df.to_csv('part_1.4_results.csv')
result_df

,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
0,DTR,5_squared_error,0.175043,0.003628,0.189747
1,DTR,20_squared_error,0.119369,0.006454,0.195066
2,DTR,20_absolute_error,0.133311,0.014386,0.194472
3,DTR,10_squared_error,0.159197,0.002112,0.198817
4,DTR,10_absolute_error,0.168505,0.006626,0.195531
5,DTR,5_absolute_error,0.182660,0.005863,0.191745
6,Ridge,1e-05,0.175918,0.005099,0.198775
7,Ridge,100000.0,0.187466,0.004458,0.186756
8,Ridge,1000.0,0.185534,0.004412,0.185520
9,Ridge,0.0001,0.175918,0.005099,0.198774


In [6]:
print('1.4.1 The hyperparameter setting for each model that resulted in the best perfomance:')
for model_names in ['DTR', 'Ridge', 'KNN']:
    display(result_df[(result_df['model_name'] == model_names)].sort_values(["mean_training_rmse", "std_training_rmse", 'test_rmse'], axis = 0, ascending = True).head(1))
print('1.4.2 The model that performed the best overall on cross validation:')
display(result_df.sort_values(["mean_training_rmse", "std_training_rmse"], axis = 0, ascending = True).head(1))
print('1.4.3 The model that performed the best overall on the final test set:')
display(result_df.sort_values(['test_rmse'], axis = 0, ascending = True).head(1))
print('1.4.5 The model that had the highest standard deviation across different fold of cross validation:')
display(result_df.sort_values(['std_training_rmse'], axis = 0, ascending = False).head(1))

1.4.1 The hyperparameter setting for each model that resulted in the best perfomance:


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
1,DTR,20_squared_error,0.119369,0.006454,0.195066


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
6,Ridge,1e-05,0.175918,0.005099,0.198775


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
17,KNN,10,0.187466,0.004458,0.186756


1.4.2 The model that performed the best overall on cross validation:


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
1,DTR,20_squared_error,0.119369,0.006454,0.195066


1.4.3 The model that performed the best overall on the final test set:


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
8,Ridge,1000.0,0.185534,0.004412,0.18552


1.4.5 The model that had the highest standard deviation across different fold of cross validation:


,model_name,hyperparameter_setting,mean_training_rmse,std_training_rmse,test_rmse
2,DTR,20_absolute_error,0.133311,0.014386,0.194472


# Part 2

In class, we have shifted from regression to classification. Here, we're going to get a little practice in optimizing one of the classification models we saw in class - logistic regression. As a reminder...

The loss function of logistic regression (also known as the logistic-loss or log-loss) is given by:
\begin{equation}
  J({\bf w}) = \frac{1}{n}\sum_{i=1}^n \log{(1 + \exp{(-y_i{\bf w}^\top{\bf x}_i}))}
  \label{eqn:logloss}
\end{equation}

The gradient for this loss function, as derived in class, is:
\begin{equation}
  \nabla J({\bf w}) = -\frac{1}{n}\sum_{i=1}^n \frac{y_i}{1 + \exp{(y_i{\bf w}^\top{\bf x}_i)}}{\bf x}_i
  \label{eqn:loglossgradient}
\end{equation}


The Hessian for the loss function is given by:
\begin{equation}
  {\bf H}({\bf w}) = \frac{1}{n} \sum_{i=1}^n \frac{\exp{(y_i{\bf w}^\top{\bf x}_i)}}{(1 + \exp{(y_i{\bf w}^\top{\bf x}_i)})^2}{\bf x}_i{\bf x}_i^\top
  \label{eqn:loglosshessian}
\end{equation}

## Part 2.1 - Logistic Regression with Gradient Descent

In Part 2.1 we will implement logistic regression with gradient descent. You need to finish implementing 3 functions:

1. `logistic_objective` - compute the logistic loss for the given data set (see equation above)
2. `logistic_gradient` - compute the gradient vector of logistic loss for the given data set (see equation above)
3. `run_gradient_descent` - run the gradient descent algorithm, given these two functions.

We have provided you with some simulation data to evaluate your method with. Part 2.1 will, however, largely be graded by evaluating your code on a slightly different dataset to ensure robustness. 

In addition, please submit answers to the following questions on your written report:

- **2.1.1** - How did you go about selecting a good step size, i.e. one that was not too big or too small? (*Note: There is more than one correct answer to this*)
- **2.1.2** - What is the condition under which we assume that the gradient descent algorithm has converged in the code here?
- **2.1.3** - What is a different convergence metric we could have used? (*Note: There is more than one correct answer to this*)

In [7]:
def logistic_objective(w, X, y):

    # compute log-loss error (scalar) with respect
    # to w (vector) for the given data X and y                               
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = scalar
    
    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    #error = 
    l1 = 0
    for i in range(len(X)):
        l1 += np.log(1+ np.exp(np.dot(-y[i],np.dot(np.transpose(w),X[i]))))
    l1 = l1/int(len(X))
    return l1

def logistic_gradient(w, X, y):

    # compute the gradient of the log-loss error (vector) with respect
    # to w (vector) for the given data X and y  
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # error = d length gradient vector (not a d x 1 matrix)

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    gradient = 0
    for i in range(len(X)):
        gradient += (y[i]/(1+np.exp(np.dot(y[i],np.dot(np.transpose(w),X[i])))))*X[i]
    gradient = -1 * (gradient/int(len(X)))
    return gradient

def run_gradient_descent(X,y):
    old_w = np.array([-1]*X.shape[1])
    # change this value! This is an unreasonable step size
    step_size = 0.0001
    new_w = old_w - 1
    
    while ((new_w-old_w)**2).sum() > .0000000001:
        s_k = logistic_gradient(old_w,X,y)
        new_w = old_w + step_size*s_k
        old_w = new_w
    return new_w

In [8]:
from scipy.stats import uniform, bernoulli
import functools
draw_binary = functools.partial(np.random.binomial,n=1)

## Simulated data to test your method
DATA_SIZE = 10000
x1 = bernoulli(.5).rvs(DATA_SIZE)
x2 = np.floor(uniform(18,60).rvs(DATA_SIZE))
true_w = [-9, 3.5, 0.2]
xb = true_w[0] + true_w[1]*x1 + true_w[2]*x2
p = 1/(1 + np.exp(-xb))
y = np.array([1 if draw_binary(p=v) else -1 for v in p])

In [9]:
from sklearn.linear_model import LogisticRegression

# notice that logistic regression as implemented in sklearn does not get the exact results either!
# so you shouldn't worry if you're a bit off
X = np.hstack([np.ones((len(xb),1)), x1[:,np.newaxis], x2[:,np.newaxis]])
model = LogisticRegression(solver='liblinear', random_state=0,fit_intercept=False)
model.fit(X,y).coef_

array([[-8.69824927,  3.30035182,  0.19495964]])

In [10]:
# this is how we will test your results
gd_result = run_gradient_descent(X,y)
# is your result relatively close to the truth?
np.abs(true_w-gd_result).sum()

13.703794209999828

## <span style="color:red"> 574 Only</span> Part 2.2 - Optimization with Newton-Raphson <span style="color:red"> 574 Only</span>

In Part 2.2, you are going to, instead of using gradient descent, use the Newton-Raphson method to optimize the same logistic regression model. To do so, you will need to 1) implement the `logistic_hessian` function to compute the Hessian matrix of logistic loss for the given data set, and 2) use `scipy`'s `optimize` function to perform the optimization, rather than writing a function by hand to do so.  

For Part 2.2, you will only need to implement these functions, we will test them using our own code. You can, however, perform the same kinds of tests that we proposed above to check your work! 

In [11]:
def logistic_hessian(w, X, y):

    # compute the Hessian of the log-loss error (matrix) with respect
    # to w (vector) for the given data X and y                               
    #
    # Inputs:
    # w = d x 1
    # X = N x d
    # y = N x 1
    # Output:
    # Hessian = d x d matrix
    
    
    if len(w.shape) == 1:
        w = w[:,np.newaxis]

    # IMPLEMENT THIS METHOD - REMOVE THE NEXT LINE
    hessian = 0
    for i in range(len(X)):
        numerator = np.exp(np.dot(y[i],np.dot(np.transpose(w),X[i])))
        denominator = (1 + np.exp(np.dot(y[i],np.dot(np.transpose(w),X[i]))))**2
        hessian += (numerator/denominator)*X[i]*np.transpose(X[i])
    hessian = hessian/int(len(X))
    return hessian

In [12]:
from scipy.optimize import minimize

def run_newton_raphson(X,y):
    args = (X,y[:,np.newaxis])
    opts = {'maxiter' : 50}    # Preferred value.    
    w_init = np.zeros(X.shape[1])
    print(w_init)
    
    # note: this is *almost* what you need, you just need to figure out what arguments are necessary here!
    soln = minimize(fun = logistic_hessian(w_init, X, y),x0 =  X.shape,
                    args=args,
                    method='Newton-CG',
                    jac = '3-point',
                    options=opts)

    w = np.transpose(np.array(soln.x))
    w = w[:,np.newaxis]
    return w

In [13]:
run_newton_raphson(X,y)

[0. 0. 0.]


ValueError: Jacobian is required for Newton-CG method